In [19]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
import seaborn as sns
sns.set(palette=sns.color_palette("husl", 8))

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1020162809.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [20]:
layer = 3
import glob
import os
path = r'layer {}'.format(layer) # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['Name']=filename.lstrip('analysis files separated layers 3 and 4/layer {}'.format(layer)).rstrip('.csv')
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame=frame[['Name','PathID','PrimaryPath','PathLength','ChildPathIDs','StartX','StartY','StartZ','EndX','EndY','EndZ']]
frame.to_csv(r'paths_raw_p21_layer{}.csv'.format(layer), index=False)

In [21]:
# Specify the layer number
layer = 3

# Standard libraries for file and OS operations
import glob
import os

# Library for data manipulation
import pandas as pd

# Set the path to the files
path = r'layer {}'.format(layer) 

# Collect all .CSV files in the given directory
all_files = glob.glob(os.path.join(path , "*.csv"))

# Initialize an empty list to hold dataframes
li = []

# Loop over each file
for filename in all_files:

    # Load each file into a DataFrame
    df = pd.read_csv(filename, index_col=None, header=0)
    
    # Add a 'Name' column to the dataframe, derived from the filename
    df['Name']=filename.lstrip('layer {}/'.format(layer)).rstrip('.csv')
    
    # Add the dataframe to the list
    li.append(df)

# Concatenate all dataframes into one
frame = pd.concat(li, axis=0, ignore_index=True)

# Keep only specific columns
frame=frame[['Name','PathID','PrimaryPath','PathLength','ChildPathIDs','StartX','StartY','StartZ','EndX','EndY','EndZ']]

# Write the final dataframe to a new .CSV file
frame.to_csv(r'paths_raw_p21_layer{}.csv'.format(layer), index=False)


In [23]:
#this part of the script extend the sheet data by adding columns for condition, IUE, mouse, dendrite type and segment lengths

paths = pd.read_csv('paths_raw_p21_layer{}.csv'.format(layer))
#calculate alternative 'length segment' length calculated as the distance between two branchpoints, different from 'PathLength' that follows the reconstrution
paths['length_segment']=np.sqrt((paths['EndZ']-paths['StartZ'])**2+(paths['EndY']-paths['StartY'])**2+(paths['EndX']-paths['StartX'])**2)
paths=paths.drop(columns=['EndZ','StartZ','EndY','StartY','EndX','StartX'])

#is creating full data for each neuron
Condition=[]
IUE=[]
Mouse=[]
dendrite_type=[]
list_name=[]
totalLengthFinal=[]
for idx, neuron in paths.iterrows():
    infos=neuron.Name.split('_')
    infos=infos[:-1]+infos[-1].split('-')
    if infos[1] == 'b':
        dendType = 'basal'
    elif infos[1] == 'a':
        dendType = 'apical'
    else:
        dendType = 'tuft'
    dendrite_type.append(dendType)
    IUE.append(infos[0])
    Condition.append('Ctrl' if infos[2][0]=='L' else 'FRMPD2B')
    Mouse.append(infos[0]+infos[2])
   # totalLengthFinal.append(totalLength[totalLength['Name']==neuron.Name]['totalLength'].iloc[0])

paths['Condition']=Condition
paths['IUE']=IUE
paths['Mouse']=Mouse
paths['dendrite_type']=dendrite_type
#paths_final['totalLength']=totalLengthFinal
#reordering columns
cols_to_order = ['Name', 'Condition','IUE','Mouse','dendrite_type']
new_columns = cols_to_order + (paths.columns.drop(cols_to_order).tolist())
paths=paths[new_columns]


In [24]:
# Recursive function to traverse a neuron's path structure
def recursive_scrolling(neuron, pathID, cursor, branch_order, method):
    # Get the child paths of the current path
    children = neuron.loc[neuron['PathID'] == pathID]['ChildPathIDs'].values[0]
    # Get the length of the current path
    length = neuron.loc[neuron['PathID'] == pathID][method].values[0]
    
    # Increment the count of the current branch order for the length of the path
    orders[int(cursor):int(cursor + length), branch_order] += 1
    # If the current path has child paths
    if type(children) is str:
        # Split the child paths into a list of IDs
        branches = [int(element) for element in children.split(',')]
        # Increase the count of the base branch order by the number of child branches
        orders[int(cursor + length):, 0] += len(branches) - 1
        # Move the cursor by the length of the current path
        cursor += length
        # Increase the branch order for child branches
        branch_order += 1
        # Recursively traverse each child branch
        for b in branches:
            recursive_scrolling(neuron, b, cursor, branch_order, method)


# Function to create a table of branch orders for a given neuron
def makeOrdersTable(orders, name, method):
    # Get the neuron data by name
    neuron = paths.query("Name=='{}'".format(name))
    # Get the primary path(s) of the neuron
    FirstPath = neuron.loc[neuron['PrimaryPath'] == True]['PathID']
    # If the dendrite type of the primary path is 'apical'
    if neuron.loc[neuron['PrimaryPath'] == True]['dendrite_type'].values[0] == 'apical':
        # Get the length of the apical dendrite
        mainApical = neuron.loc[neuron['PrimaryPath'] == True][method].values[0]
        # Start recursive scrolling from the negative of the length of the apical dendrite to count distance from the first oblique branch
        recursive_scrolling(neuron, FirstPath.values[0], -mainApical, 0, method)
    else:
        # Start recursive scrolling for each primary path from the start
        for multipleFirst in FirstPath:
            recursive_scrolling(neuron, multipleFirst, 0, 1, method)
    
    # Initialize a list to store the average branch order at each distance
    average_order = []
    # Calculate the average branch order at each distance
    for i in range(len(orders)):
        sum_order_dist = 0
        number_branches = 0
        for j in range(1, len(orders[i, :])):    
            sum_order_dist += orders[i, j] * j
            number_branches += orders[i, j]
        average_order.append(sum_order_dist / number_branches if number_branches != 0 else 0)

    # Add the average order data as a new column to the orders array
    orders = np.column_stack((np.arange(1, sizemax + 1, dtype=int), orders, average_order))

    return orders


In [25]:
methods=['length_segment','PathLength']
for i in range(2):
    method_number=i
    method=methods[method_number]
    #creating a DataFrame with the path length analysis for each neuron
    names=paths['Name'].drop_duplicates().values
    paths_final=pd.DataFrame(columns = ['Name','Distance','cumulative_nodes','First_order','Second_order','Third_order','Fourth_order','Fifth_order','Sixth_order','Seventh_order','Eighth_order','Ninth_order','Average_order'])
    sizemax=300
    ordermax=9
    sumNeuron=[] #calculates the total length of dendrites in one neuron
    for name in names:
        somme=paths[paths['Name']==name]['PathLength'].sum()
        sumNeuron.append(somme)
        orders=np.zeros((sizemax,ordermax+1))
        table=np.column_stack(([name]*sizemax,makeOrdersTable(orders,name,method)))
        sub_df = pd.DataFrame(table, # new neuron
                              columns = paths_final.columns)                               
        paths_final=paths_final.append(sub_df)
    
    totalLength= pd.DataFrame(np.transpose([names,np.array(sumNeuron)]),
                                  columns=['Name', 'totalLength'])
    #is creating full data for each neuron
    Condition=[]
    IUE=[]
    Mouse=[]
    dendrite_type=[]
    list_name=[]
    totalLengthFinal=[]
    for idx, neuron in paths_final.iterrows():
        infos=neuron.Name.split('_')
        infos=infos[:-1]+infos[-1].split('-')
        if infos[1] == 'b':
            dendType = 'basal'
        elif infos[1] == 'a':
            dendType = 'apical'
        else:
            dendType = 'tuft'
        dendrite_type.append(dendType)
        IUE.append(infos[0])
        Condition.append('Ctrl' if infos[2][0]=='L' else 'FRMPD2B')
        Mouse.append(infos[0]+infos[2])
        totalLengthFinal.append(totalLength[totalLength['Name']==neuron.Name]['totalLength'].iloc[0])

    paths_final['Condition']=Condition
    paths_final['IUE']=IUE
    paths_final['Mouse']=Mouse
    paths_final['dendrite_type']=dendrite_type
    paths_final['totalLength']=totalLengthFinal
    #reordering columns
    cols_to_order = ['Name', 'Condition','IUE','Mouse','dendrite_type','totalLength']
    new_columns = cols_to_order + (paths_final.columns.drop(cols_to_order).tolist())
    paths_final=paths_final[new_columns]
    
    paths_final.to_csv(r'paths_ready_p21_{}_layer{}.csv'.format(method,layer), index=False)

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=pat

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=pat

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=pat

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=pat

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=pat

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=pat

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1083811878.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final=pat

In [26]:
# List of methods for path length analysis
methods=['length_segment','PathLength']

# Loop over each method
for i in range(2):
    method_number = i
    method = methods[method_number]

    # Create a DataFrame to store the path length analysis for each neuron
    names = paths['Name'].drop_duplicates().values
    paths_final = pd.DataFrame(columns = ['Name','Distance','cumulative_nodes','First_order','Second_order',
                                          'Third_order','Fourth_order','Fifth_order','Sixth_order','Seventh_order',
                                          'Eighth_order','Ninth_order','Average_order'])

    sizemax = 300
    ordermax = 9
    sumNeuron = [] # To store the total length of dendrites in one neuron

    for name in names:
        somme = paths[paths['Name'] == name]['PathLength'].sum()
        sumNeuron.append(somme)
        orders = np.zeros((sizemax, ordermax + 1))
        table = np.column_stack(([name]*sizemax, makeOrdersTable(orders, name, method)))

        # Append the neuron's path analysis to the final DataFrame
        sub_df = pd.DataFrame(table, columns = paths_final.columns)                               
        paths_final = paths_final.append(sub_df)

    # Create a DataFrame to store the total dendrite length for each neuron
    totalLength = pd.DataFrame(np.transpose([names, np.array(sumNeuron)]), columns=['Name', 'totalLength'])

    # Initialize lists to store neuron information
    Condition = []
    IUE = []
    Mouse = []
    dendrite_type = []
    list_name = []
    totalLengthFinal = []

    for idx, neuron in paths_final.iterrows():
        # Parse the neuron's name to extract information
        infos = neuron.Name.split('_')
        infos = infos[:-1] + infos[-1].split('-')
        if infos[1] == 'b':
            dendType = 'basal'
        elif infos[1] == 'a':
            dendType = 'apical'
        else:
            dendType = 'tuft'

        dendrite_type.append(dendType)
        IUE.append(infos[0])
        Condition.append('Ctrl' if infos[2][0] == 'L' else 'FRMPD2B')
        Mouse.append(infos[0] + infos[2])
        totalLengthFinal.append(totalLength[totalLength['Name'] == neuron.Name]['totalLength'].iloc[0])

    # Add the neuron information as new columns to the final DataFrame
    paths_final['Condition'] = Condition
    paths_final['IUE'] = IUE
    paths_final['Mouse'] = Mouse
    paths_final['dendrite_type'] = dendrite_type
    paths_final['totalLength'] = totalLengthFinal

    # Reorder the columns of the final DataFrame
    cols_to_order = ['Name', 'Condition','IUE','Mouse','dendrite_type','totalLength']
    new_columns = cols_to_order + (paths_final.columns.drop(cols_to_order).tolist())
    paths_final = paths_final[new_columns]
    
    # Save the final DataFrame to a .csv file
    paths_final.to_csv(r'paths_ready_p21_allnew_{}_layer{}.csv'.format(method, layer), index = False)


/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_fin

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_fin

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_fin

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_fin

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_fin

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_fin

/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_final = paths_final.append(sub_df)
/var/folders/9d/r0b5fdcn1vv81f_fr6lkyq6h0000gp/T/ipykernel_3505/1237365405.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  paths_fin

In [27]:
# Define the layer and the methods
layer = 3
methods=['length_segment','PathLength']
# Specify the IUE values of interest
IUE = ["19","76","181","184"] 

# Create an empty list to store the data
data = []

# Loop over each method ('length_segment' and 'PathLength')
for method in methods:
    # Load data from the csv file
    df = pd.read_csv('paths_ready_p21_{}_layer{}.csv'.format(method, layer))
    
    # Convert the 'IUE' column to string type for compatibility with the 'IUE' list
    df['IUE'] = df['IUE'].astype(str) 
    
    # Loop over 'dendriteType' and filter the data accordingly
    for dendriteType in ['apical', 'basal']:
        # Filter the data based on 'dendriteType' and 'IUE' values
        df_filtered = df.query("dendrite_type==@dendriteType and IUE in @IUE")
        
        # Append the filtered DataFrame to the 'data' list
        data.append(df_filtered)
    
        # Loop over each condition ('Ctrl' and 'FRMPD2B')
        for cond in ["Ctrl","FRMPD2B"]:
            # Filter the DataFrame based on the condition
            df_cond = df_filtered.query("Condition ==@cond")
            
            # Get the unique neuron names
            neurones = df_cond['Name'].unique()
            
            # Create a new DataFrame to store the data for each neuron
            new_df = pd.DataFrame()
            
            # Loop over each neuron
            for neuron in neurones:
                # Get the 'cumulative_nodes' values for the current neuron
                values = df_cond.query("Name == @neuron")['cumulative_nodes'].tolist()
                
                # Add a new column in 'new_df' with the neuron name and fill it with the 'cumulative_nodes' values
                new_df[neuron] = values
            
            # Write the DataFrame to an Excel file
            new_df.to_excel(r'cumulBranchingPerNeuron_{}_p21_{}_layer{}.xlsx'.format(cond,dendriteType,layer), index=False)
